# EDA on Loan problem of minimising defaulters

## Basic setup

In [0]:
# Load libraries
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

### Mounting GDrive locally


In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
# Load the data
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Practice/Loan problem/loan.csv')
df.head()

In [0]:
df.info()

In [0]:
df.describe()

In [0]:
# Remove id columns as we don't required them
df = df.drop('id', axis=1)
df = df.drop('member_id', axis=1)

In [0]:
def missing_values_percent(df_param):
  print(100*round(df_param.isnull().sum()/len(df_param.index),2))

In [0]:
missing_values_percent(df)

In [0]:
# drop columns with more than 30% missing values
threshold = len(df.index)*0.3
df_cleaned = df.dropna(axis=1, thresh=threshold)
missing_values_percent(df_cleaned)

In [0]:
df_cleaned = df_cleaned.drop('desc', axis=1)
df_cleaned = df_cleaned.drop('mths_since_last_delinq', axis=1)
missing_values_percent(df_cleaned)

In [0]:
# rows with more than 5 missing values
df_cleaned[df_cleaned.isnull().sum(axis=1)>5] # 0 rows

In [0]:
# drop rows with any NA values
df_cleaned  = df_cleaned.dropna(axis=0)
missing_values_percent(df_cleaned)

In [0]:
df_cleaned.info()

In [0]:
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
def format_datetime_columns(df_param, col_name):
  df_param[col_name] = df_param[col_name].apply(lambda x: '01-'+x)
  df_param[col_name] = pd.to_datetime(df_param[col_name], format='%d-%b-%y')

In [0]:
format_datetime_columns(df_cleaned, 'issue_d')
format_datetime_columns(df_cleaned, 'earliest_cr_line')
format_datetime_columns(df_cleaned, 'last_pymnt_d')
format_datetime_columns(df_cleaned, 'last_credit_pull_d')

In [0]:
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
def remove_suffix_from_columns(df_param, col_name, suffix_to_remove, is_numeric = False):
  new_col_name=col_name+'_in_'+suffix_to_remove
  df_param[new_col_name] = df_param[col_name].str.rstrip(suffix_to_remove)
  if is_numeric:
    df_param[new_col_name] = pd.to_numeric(df_param[new_col_name])
  df_param.drop(col_name, axis=1, inplace=True)

In [0]:
remove_suffix_from_columns(df_cleaned, 'term', ' months', is_numeric=True)
remove_suffix_from_columns(df_cleaned, 'int_rate', '%', is_numeric=True)
remove_suffix_from_columns(df_cleaned, 'revol_util', '%', is_numeric=True)

In [0]:
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
# Fix emp_length column
df_cleaned['emp_length'] = df_cleaned['emp_length'].str.replace('years', 'year')
df_cleaned['emp_length'] = df_cleaned['emp_length'].str.replace(' year', '')
df_cleaned['emp_length'] = df_cleaned['emp_length'].str.replace('\\< 1', '-1')
df_cleaned['emp_length'] = df_cleaned['emp_length'].str.replace('10\\+', '100')
df_cleaned['emp_length'] = pd.to_numeric(df_cleaned['emp_length'])

In [0]:
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
# drop sub_grade column 
df_cleaned = df_cleaned.drop('sub_grade', axis=1)

In [0]:
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
# check url column relevancy
df_cleaned['url'][1] # 'https://lendingclub.com/browse/loanDetail.action?loan_id=1077430'


In [0]:
# Since url contians only Loan id, we can drop the url column
df_cleaned = df_cleaned.drop('url', axis=1)

In [0]:
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
# convert all strings to lowercase for object columns
str_columns = ['grade','emp_title','home_ownership','verification_status','loan_status','pymnt_plan','purpose','title','zip_code','addr_state','initial_list_status','application_type']
df_cleaned[str_columns] = df_cleaned[str_columns].apply(lambda x: x.astype(str).str.lower())
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
# check unique values for each column
print(df_cleaned['grade'].value_counts())
print(df_cleaned['emp_title'].value_counts())
print(df_cleaned['home_ownership'].value_counts())
print(df_cleaned['verification_status'].value_counts())
print(df_cleaned['loan_status'].value_counts())
print(df_cleaned['pymnt_plan'].value_counts())
print(df_cleaned['purpose'].value_counts())
print(df_cleaned['title'].value_counts())
print(df_cleaned['zip_code'].value_counts())
print(df_cleaned['addr_state'].value_counts())
print(df_cleaned['initial_list_status'].value_counts())
print(df_cleaned['application_type'].value_counts())

In [0]:
# drop emp_title as variation is high
# drop pymnt_plan all values = 'n'
# drop title as variation is high
# drop initial_list_status all values = 'f'
# drop application_type all values = 'individual'

df_cleaned = df_cleaned.drop(['emp_title','pymnt_plan','title','zip_code','addr_state','initial_list_status','application_type'], axis=1)

In [0]:
df_cleaned.loc[:, df_cleaned.dtypes==object].head()

In [0]:
# Since we want to consider entries only if it 'charged_off' or 'fully paid', remove rows with 'current' as loan_status
df_cleaned = df_cleaned[df_cleaned['loan_status'] != 'current']
df_cleaned.info()